## 원본

In [30]:
import requests
from bs4 import BeautifulSoup

URL = 'https://finance.naver.com/sise/sise_quant.nhn'
res = requests.get(URL)
# print(res.status_code)

soup = BeautifulSoup(res.text, 'html.parser')
stocks = soup.select('.type_2 tr')[2:]

codes = []

for stock in stocks:
    try:
        stock_n = stock.select_one('.tltle').text
        price = stock.select_one('.number').text.replace(',', '')
        code = stock.select_one('.tltle').attrs['href'][-6:]

        if (('인버스' not in stock_n) and ('레버리지' not in stock_n)) and (int(price) <= 26000):
            codes.append(code)
        else:
            continue
    except:
        pass
        
    if len(codes) == 5:
        break
        
print(codes)

['001570', '009270', '006220', '010580', '096300']


## 작업 중.. 
### (codes에 담긴 종목 중 변동성 돌파 기반으로 매수해야 할 종목이 있는지 추가적으로 데이터 크롤링 후 시각화 예정)

In [3]:
import requests
from bs4 import BeautifulSoup

URL = 'https://finance.naver.com/sise/sise_quant.nhn'
res = requests.get(URL)
# print(res.status_code)

soup = BeautifulSoup(res.text, 'html.parser')
stocks = soup.select('.type_2 tr')[2:]

In [4]:
stocks[0].select('.tah.p11')

[<span class="tah p11 red02">
 				30
 				</span>,
 <span class="tah p11 red01">
 				+0.95%
 				</span>]

In [5]:
stocks[0].select('.tah.p11')[0].text

'\n\t\t\t\t30\n\t\t\t\t'

In [6]:
stocks[0].select('.tah.p11')[1].text

'\n\t\t\t\t+0.95%\n\t\t\t\t'

In [7]:
import re

In [8]:
a = stocks[0].select('.tah.p11')[0].text
b = stocks[0].select('.tah.p11')[1].text
re_a = re.sub('[a-zA-Z]', '', a)
re_b = re.sub('[a-zA-Z]', '', b)
print(re_a, re_b)


				30
				 
				+0.95%
				


In [9]:
Notag_a = re.sub('\t*|\n*|(\<[^\>]*\>)|\n*\t*', '', a)
Notag_b = re.sub('\t*|\n*|(\<[^\>]*\>)|\n*\t*', '', b)
print(Notag_a, Notag_b)

30 +0.95%


In [36]:
ccc = stocks[1].select('td.number')[3:-2]
print(ccc, stocks[1].select('td.number')[3].text, stocks[1].select('td.number')[-3].text, sep = '\n')
ccc[0].text

[<td class="number">63,913,877</td>, <td class="number">33,485</td>, <td class="number">505</td>, <td class="number">506</td>, <td class="number">1,326</td>]
63,913,877
1,326


'63,913,877'

In [35]:
pp = re.sub('\t*|\n*|(\<[^\>]*\>)|\n*\t*|, ', '', str(ccc[0]))
pp

'63,913,877'

In [14]:
print(type(ccc[0]))

<class 'bs4.element.Tag'>


In [15]:
result = []

for cc in ccc:
    c = re.sub('\t*|\n*|(\<[^\>]*\>)|\n*\t*|, ', '', str(cc))
    print(cc)
    result.append(c)
result # 거래량, 거래대금, 매수호가, 매도호가, 시가총액

<td class="number">63,913,877</td>
<td class="number">33,485</td>
<td class="number">505</td>
<td class="number">506</td>
<td class="number">1,326</td>


['63,913,877', '33,485', '505', '506', '1,326']

In [32]:
import requests
from bs4 import BeautifulSoup
import re

URL = 'https://finance.naver.com/sise/sise_quant.nhn'
res = requests.get(URL)
# print(res.status_code)

soup = BeautifulSoup(res.text, 'html.parser')
stocks = soup.select('.type_2 tr')[2:]

codes = []
keys = ['종목명', '현재가', '전일비', '등락률', '거래량', '시가총액']
result = []

for stock in stocks:
    try:
        stock_n = stock.select_one('.tltle').text
        price = stock.select_one('.number').text.replace(',', '')

        if (('인버스' not in stock_n) and ('레버리지' not in stock_n)) and (int(price) <= 26000):
            
            code = stock.select_one('.tltle').attrs['href'][-6:]
            codes.append(code)
            
            updown = stock.select('.tah.p11')[0].text
            percentage = stock.select('.tah.p11')[1].text
            
            re_updown = re.sub('\t*|\n*|(\<[^\>]*\>)|\n*\t*', '', updown)
            re_percentage = re.sub('\t*|\n*|(\<[^\>]*\>)|\n*\t*', '', percentage)
            
            market = stock.select('td.number')[3].text
            volumn = stock.select('td.number')[-3].text
            
            values = [stock_n, price, re_updown, re_percentage, market, volumn]
            
            info_dict = dict(zip(keys, values))
            result.append(info_dict)
        
        else:
            continue
    except:
        pass
        
    if len(codes) == 5:
        break
        
print(codes)

for r in result:
    print()
    print(r)

['001570', '009270', '006220', '010580', '096300']

{'종목명': '금양', '현재가': '17450', '전일비': '600', '등락률': '+3.56%', '거래량': '25,566,113', '시가총액': '10,048'}

{'종목명': '신원', '현재가': '1680', '전일비': '55', '등락률': '+3.38%', '거래량': '16,567,236', '시가총액': '1,606'}

{'종목명': '제주은행', '현재가': '7020', '전일비': '730', '등락률': '+11.61%', '거래량': '11,798,306', '시가총액': '2,255'}

{'종목명': '에스엠벡셀', '현재가': '2700', '전일비': '405', '등락률': '+17.65%', '거래량': '11,708,144', '시가총액': '3,004'}

{'종목명': '베트남개발1', '현재가': '71', '전일비': '5', '등락률': '+7.58%', '거래량': '11,487,373', '시가총액': '98'}


In [33]:
import requests
from bs4 import BeautifulSoup
import re

URL = 'https://finance.naver.com/sise/sise_quant.nhn'
res = requests.get(URL)
# print(res.status_code)

soup = BeautifulSoup(res.text, 'html.parser')
stocks = soup.select('.type_2 tr')[2:]

codes = []
keys = ['종목명', '현재가', '전일비', '등락률', '거래량', '시가총액']
result = []
urls = []

for stock in stocks:
    try:
        company = stock.select_one('.tltle').text
        price = stock.select_one('.number').text.replace(',', '')

        if (('인버스' not in company) and ('레버리지' not in company)) and (int(price) <= 26000):
            
            code = stock.select_one('.tltle').attrs['href'][-6:] # 종목코드 가져오기
            codes.append(code)
            
            updown = stock.select('.tah.p11')[0].text # 전일비 가져오기
            percentage = stock.select('.tah.p11')[1].text # 등락률 가져오기
            
            re_updown = re.sub('\t*|\n*|(\<[^\>]*\>)|\n*\t*', '', updown) # 전일비 정규표현식으로 클렌징
            re_percentage = re.sub('\t*|\n*|(\<[^\>]*\>)|\n*\t*', '', percentage) # 등락률 정규표현식으로 클렌징
            
            market = stock.select('td.number')[3].text # 거래량 가져오기
            volumn = stock.select('td.number')[-3].text # 시가총액 가져오기
            
            values = [company, price, re_updown, re_percentage, market, volumn]
            
            info_dict = dict(zip(keys, values))
            result.append(info_dict)
            
            sise_URL = 'https://finance.naver.com/item/sise_day.naver?code=' + code # 각 종목의 시세 탭으로 진입
            urls.append(sise_URL)
#             sise_res = requests.get(sise_URL)
        
        else:
            continue
    except:
        pass
        
    if len(codes) == 5:
        break
        
print(codes, [type(i) for i in codes])

for r in result:
    print()
    print(r)

print(*urls, sep='\n')

# for url in urls:
#     res = requests.get(url)
#     soup = BeautifulSoup(res.text, 'html.parser')

['001570', '009270', '006220', '010580', '096300'] [<class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>]

{'종목명': '금양', '현재가': '17450', '전일비': '600', '등락률': '+3.56%', '거래량': '25,566,113', '시가총액': '10,048'}

{'종목명': '신원', '현재가': '1680', '전일비': '55', '등락률': '+3.38%', '거래량': '16,567,236', '시가총액': '1,606'}

{'종목명': '제주은행', '현재가': '7020', '전일비': '730', '등락률': '+11.61%', '거래량': '11,798,306', '시가총액': '2,255'}

{'종목명': '에스엠벡셀', '현재가': '2700', '전일비': '405', '등락률': '+17.65%', '거래량': '11,708,144', '시가총액': '3,004'}

{'종목명': '베트남개발1', '현재가': '71', '전일비': '5', '등락률': '+7.58%', '거래량': '11,487,373', '시가총액': '98'}
https://finance.naver.com/item/sise_day.naver?code=001570
https://finance.naver.com/item/sise_day.naver?code=009270
https://finance.naver.com/item/sise_day.naver?code=006220
https://finance.naver.com/item/sise_day.naver?code=010580
https://finance.naver.com/item/sise_day.naver?code=096300


## 고정 돌파계수 K vs. 시장 상황(전날 추세 - 9/8 목)에 따른 변동 돌파계수 K

#### 날짜
#### 종가
#### 전일비
#### 시가
#### 고가
#### 저가
#### 거래량
 
#### 특정 종목의 (전일 고가-전일 저가)가 갑자기 크게 증가한 다음날 (상한가를 치거나 주가가 많이 오른 경우) 
#### 전날보다는 못하지만 어느 정도 상승장을 이어가고 있을 때 고정된 돌파계수 0.5 때문에 매수 타이밍을 못 잡는 경우가 생길 수도 있고 
#### 반대의 경우, 즉 (전일 고가-전일 저가)가 작았던 다음날 작은 주가 상승에도 매수를 시도하는 경우가 생기기도 함

In [27]:
import requests
from bs4 import BeautifulSoup

headers = {'user-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:104.0) Gecko/20100101 Firefox/104.0'}
sise_URL = 'https://finance.naver.com/item/sise_day.naver?code=001570'
sise_res = requests.get(sise_URL, headers=headers)
# print(res.status_code)

soup = BeautifulSoup(sise_res.text, 'html.parser')
prices = soup.select('.type2 tr')[2]

latest_date = prices.select_one('.tah.p10.gray03').text # 최근 날짜
start_price = int(prices.select('.tah.p11')[2].text.replace(',', '')) # 전날 시가이기 때문에 필요 없음. 당일 시가로 대체해야
end_price = int(prices.select('.tah.p11')[0].text.replace(',', '')) # 종가
highest_price = int(prices.select('.tah.p11')[3].text.replace(',', '')) # 고가
lowest_price = int(prices.select('.tah.p11')[4].text.replace(',', '')) # 저가

# K = 0.5 # 고정 돌파계수 K
K = 1 - (abs(start_price - end_price) / (highest_price - lowest_price)) # 돌파계수 K, 9/8 - 0.85

target_price = start_price + (highest_price - lowest_price) * K # 매수 타켓가

print(prices, f'\n최근 날짜: {latest_date}, \n시가: {start_price}, \n종가: {end_price}, \n고가: {highest_price}, \n저가: {lowest_price}, \n돌파계수 K: {K}, \n매수 타겟가: {target_price}')

fixedK = start_price + (highest_price - lowest_price) * 0.5 # 돌파계수 K가 default 0.5일 때. 매수 타켓가: 19200
flexibleK = start_price + (highest_price - lowest_price) * K # 돌파계수 K가 시장 상황에 따라 0.85일 때. 매수 타켓가: 20950
print(fixedK, flexibleK, f'차이: {flexibleK} - {fixedK} =', abs(flexibleK - fixedK))

<tr onmouseout="mouseOut(this)" onmouseover="mouseOver(this)">
<td align="center"><span class="tah p10 gray03">2022.09.08</span></td>
<td class="num"><span class="tah p11">17,450</span></td>
<td class="num">
<img alt="상승" height="6" src="https://ssl.pstatic.net/imgstock/images/images4/ico_up.gif" style="margin-right:4px;" width="7"/><span class="tah p11 red02">
				600
				</span>
</td>
<td class="num"><span class="tah p11">16,700</span></td>
<td class="num"><span class="tah p11">21,250</span></td>
<td class="num"><span class="tah p11">16,250</span></td>
<td class="num"><span class="tah p11">25,566,113</span></td>
</tr> 
최근 날짜: 2022.09.08, 
시가: 16700, 
종가: 17450, 
고가: 21250, 
저가: 16250, 
돌파계수 K: 0.85, 
매수 타겟가: 20950.0
19200.0 20950.0 차이: 20950.0 - 19200.0 = 1750.0


## 1. 시장 상황(추세)에 따른 돌파계수 K를 구하기 귀해 열흘 치 주가 정보 가져와서 평균을 구하자

## 2. 열흘 치 주가 정보를 고정/변동/평균 K로 운영했을 때 예상 수익을 비교해보자 (백테스트)
- ### 고정 K: -200, 변동 K: -401, 변동 K 평균: -1168

In [98]:
import requests
from bs4 import BeautifulSoup

headers = {'user-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:104.0) Gecko/20100101 Firefox/104.0'}
sise_URL = 'https://finance.naver.com/item/sise_day.naver?code=001570'
sise_res = requests.get(sise_URL, headers=headers)
# print(res.status_code)

soup = BeautifulSoup(sise_res.text, 'html.parser')
prices = soup.select('.type2 tr')[2:-1] # 최근 열흘치 주가 정보 가져오기
# print(prices[-1].text)

fixedKprofits = 0
flexibleKprofits = 0
flexibleKs = 0

for price in prices:
#     print(price.text)
    
    try:
        latest_date = price.select_one('.tah.p10.gray03').text # 최근 날짜
        start_price = int(price.select('.tah.p11')[2].text.replace(',', '')) # 전날 시가이기 때문에 필요 없음. 당일 시가로 대체해야
        end_price = int(price.select('.tah.p11')[0].text.replace(',', '')) # 종가
        highest_price = int(price.select('.tah.p11')[3].text.replace(',', '')) # 고가
        lowest_price = int(price.select('.tah.p11')[4].text.replace(',', '')) # 저가
        
        K = 1 - (abs(start_price - end_price) / (highest_price - lowest_price)) # 돌파계수 K, 9/8 - 0.85
        K = round(K, 2)
        flexibleKs += K # 변동 K의 평균 구하기

        fixedKtarget = int(start_price + (highest_price - lowest_price) * 0.5) # 돌파계수 K가 default 0.5일 때 매수 타켓가
        flexibleKtarget = int(start_price + (highest_price - lowest_price) * K) # 돌파계수 K가 시장 상황에 따라 0.85일 때 매수 타켓가

        print(f'날짜: {latest_date}, \n시가: {start_price}, \n종가: {end_price}, \n고가: {highest_price}, \n저가: {lowest_price}, \
              \n변동 돌파계수 K: {K}')
        
        gap = abs(flexibleKtarget - fixedKtarget)
        
        print(f'차이: 변동 K {flexibleKtarget} - 고정 K {fixedKtarget} =', gap)
        
        # 현재가가 매수 타겟가를 넘어서자 마자 매수할 것이고, 종가에 근접한 가격으로 매도할 것이기 때문에 데이 트레이딩 수익 예측이 가능함
        BuyOrDont = highest_price >= flexibleKtarget # 매수 여부 확인
        
        if BuyOrDont == True: # 매수 했을 경우 수익 계산
            fixedKprofit = end_price - fixedKtarget # 고정 K 예상 수익
            flexibleKprofit = end_price - flexibleKtarget # 변동 K 예상 수익

            fixedKprofits += fixedKprofit # 고정 K 예상 누적 수익
            flexibleKprofits += flexibleKprofit # 변동 K 예상 누적 수익
        
            if flexibleKprofit > fixedKprofit:
                versus = '변동 K 승리'

            elif flexibleKprofit < fixedKprofit:
                versus = '고정 K 승리'
                
        else:
            fixedKprofit = 0 # 고정 K 예상 수익
            flexibleKprofit = 0 # 변동 K 예상 수익
            versus = '동점'
        
        print(f'매수 여부(변동 K): {BuyOrDont}')
        print(f'고정 K 예상 수익: {fixedKprofit}, 변동 K 예상 수익: {flexibleKprofit}, 승부: {versus}\n')
        
    except:
        continue
    
print(f'고정 K로 열흘 운용했을 때 예상 수익: {fixedKprofits}, 변동 K로 열흘 운용했을 때 예상 수익: {flexibleKprofits}')
# 고정 K로 열흘 운용했을 때 예상 수익: -200, 변동 K로 열흘 운용했을 때 예상 수익: -401, 변동 K 평균으로 열흘 운용했을 때 예상 수익: -1168

if fixedKprofits < 0 and flexibleKprofits < 0:
    print('경기가 안 좋다.. 하락장!\n')
    
elif fixedKprofits <= 1000 and flexibleKprofits <= 1000:
    print('경기가 그냥 그렇다. 보합권!\n')
    
else:
    print('경기가 좋다! 상승장!\n')
    
print(f'변동 K의 열흘 치 평균: {round(flexibleKs / 10, 2)}')

날짜: 2022.09.08, 
시가: 16700, 
종가: 17450, 
고가: 21250, 
저가: 16250,               
변동 돌파계수 K: 0.85
차이: 변동 K 20950 - 고정 K 19200 = 1750
매수 여부(변동 K): True
고정 K 예상 수익: -1750, 변동 K 예상 수익: -3500, 승부: 고정 K 승리

날짜: 2022.09.07, 
시가: 15200, 
종가: 16850, 
고가: 16850, 
저가: 14950,               
변동 돌파계수 K: 0.13
차이: 변동 K 15447 - 고정 K 16150 = 703
매수 여부(변동 K): True
고정 K 예상 수익: 700, 변동 K 예상 수익: 1403, 승부: 변동 K 승리

날짜: 2022.09.06, 
시가: 12950, 
종가: 14850, 
고가: 15650, 
저가: 12900,               
변동 돌파계수 K: 0.31
차이: 변동 K 13802 - 고정 K 14325 = 523
매수 여부(변동 K): True
고정 K 예상 수익: 525, 변동 K 예상 수익: 1048, 승부: 변동 K 승리

날짜: 2022.09.05, 
시가: 12600, 
종가: 13150, 
고가: 13550, 
저가: 11350,               
변동 돌파계수 K: 0.75
차이: 변동 K 14250 - 고정 K 13700 = 550
매수 여부(변동 K): False
고정 K 예상 수익: 0, 변동 K 예상 수익: 0, 승부: 동점

날짜: 2022.09.02, 
시가: 12150, 
종가: 12100, 
고가: 12250, 
저가: 11800,               
변동 돌파계수 K: 0.89
차이: 변동 K 12550 - 고정 K 12375 = 175
매수 여부(변동 K): False
고정 K 예상 수익: 0, 변동 K 예상 수익: 0, 승부: 동점

날짜: 2022.09.01, 
시가: 11700, 
종가: 12100,